<a href="https://colab.research.google.com/github/franklaercio/paul-the-octopus/blob/main/model/paultheoctopus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paul The Octopus Machine Learning

Code by Frank Laércio, Software Developer at CI&T.<br>
frank.junior@ciandt.com [github.com/franklaercio](https://github.com/franklaercio)

![banner.png](https://drive.google.com/uc?export=view&id=1euYvRBbkQtYMslUTwaUUdgcGFc_1gupL)

Content

1.   Importing libraries and files for GCP
2.   Fifa World Cup Data Analyzing



## **Importing libraries and files for GCP**

In this section we are import all libraries that need on this project.

*   Numpy for linear algebra
*   Pandas for manipuling data
*   Seaborn for data visualization
*   Matplotlib for data visualization



In [1]:
import numpy as np 
import pandas as pd
import re
import calendar

import seaborn as sns
sns.set_theme(style="darkgrid")

import matplotlib.pyplot as plt

from google.colab import auth
auth.authenticate_user()

project_id = 'phoenix-cit'
!gcloud config set project {project_id}

Updated property [core/project].


We are now starting to download files from Google Cloud Provider (GCP) and save to our Colab Notebook.

In [4]:
!gsutil -m cp gs://paul-the-octopus-frank-junior/*.csv /content/files

Copying gs://paul-the-octopus-frank-junior/historical-results.csv...
Copying gs://paul-the-octopus-frank-junior/shootouts.csv...
Copying gs://paul-the-octopus-frank-junior/historical_win-loose-draw_ratios.csv...
Copying gs://paul-the-octopus-frank-junior/matches-schedule.csv...
Copying gs://paul-the-octopus-frank-junior/ranking.csv...
Copying gs://paul-the-octopus-frank-junior/sample_predictions_submission.csv...
\ [6/6 files][  5.9 MiB/  5.9 MiB] 100% Done                                    
Operation completed over 6 objects/5.9 MiB.                                      


## **Fifa World Cup Data Analyzing**

In this section, we understand our dataset and make some assumptions. Like a does the FIFA ranking influence the results, are the clashes in world cups important in today's results or the win set is an interesting data.

First, we need to import the entire dataset for some assumptions.

In [121]:
df_historical_results = pd.read_csv('/content/files/historical-results.csv')
df_win_loose_draw = pd.read_csv('/content/files/historical_win-loose-draw_ratios.csv')
df_ranking = pd.read_csv('/content/files/ranking.csv')

We need understand the import data.

In [80]:
df_historical_results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False


In [79]:
df_historical_results.columns

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'city', 'country', 'neutral'],
      dtype='object')

In [81]:
for col in ['date']:
    df_historical_results[col] = pd.to_datetime(df_historical_results[col], format='%Y-%m-%d', errors='ignore')

In [82]:
df_historical_results.isnull().sum()

date          0
home_team     0
away_team     0
home_score    1
away_score    1
tournament    0
city          0
country       0
neutral       0
dtype: int64

In [106]:
df_historical_results['id'] = df_historical_results.index

In [118]:
df_historical_results.loc[:, ['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'city', 'country', 'neutral', 'id']]

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,id
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False,0
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False,1
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False,2
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False,3
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False,4
...,...,...,...,...,...,...,...,...,...,...
44045,2022-09-27,Qatar,Chile,2.0,2.0,Friendly,Vienna,Austria,True,44045
44046,2022-09-27,Saudi Arabia,United States,0.0,0.0,Friendly,Murcia,Spain,True,44046
44047,2022-09-27,South Africa,Botswana,1.0,0.0,Friendly,Johannesburg,South Africa,False,44047
44048,2022-09-27,United Arab Emirates,Venezuela,0.0,4.0,Friendly,Wiener Neustadt,Austria,True,44048


In [85]:
# Removing collumns not using in the model

df_historical_results = df_historical_results.drop (['neutral'], axis = 1)
df_historical_results = df_historical_results.drop (['city'], axis = 1)

In [122]:
#pd.unique(df_historical_results['tournament'])
frequently_tornament = df_historical_results['tournament'].value_counts().index.tolist()
frequently_tornament

['Friendly',
 'FIFA World Cup qualification',
 'UEFA Euro qualification',
 'African Cup of Nations qualification',
 'FIFA World Cup',
 'Copa América',
 'AFC Asian Cup qualification',
 'African Cup of Nations',
 'CECAFA Cup',
 'CFU Caribbean Cup qualification',
 'Merdeka Tournament',
 'British Championship',
 'UEFA Nations League',
 'Gulf Cup',
 'AFC Asian Cup',
 'Gold Cup',
 'Island Games',
 'UEFA Euro',
 'COSAFA Cup',
 'AFF Championship',
 'Nordic Championship',
 'African Nations Championship',
 'CFU Caribbean Cup',
 "King's Cup",
 'Amílcar Cabral Cup',
 'South Pacific Games',
 'CONCACAF Nations League',
 'CONCACAF Championship',
 'UNCAF Cup',
 'Korea Cup',
 'CONCACAF Championship qualification',
 'SAFF Cup',
 'Arab Cup',
 'Confederations Cup',
 'International Cup',
 'CCCF Championship',
 'EAFF Championship',
 'CONIFA World Football Cup',
 'Windward Islands Tournament',
 'Oceania Nations Cup',
 'AFC Challenge Cup',
 'WAFF Championship',
 'Baltic Cup',
 'AFC Challenge Cup qualification

In [116]:
#df_historical_results.drop(df_historical_results.index[(df_historical_results['tournament'] != 'FIFA World Cup')],axis=0,inplace=True)
df_historical_results = df_historical_results[df_historical_results['tournament'].isin(frequently_tornament)]

In [117]:
df_historical_results.count()

date          34197
home_team     34197
away_team     34197
home_score    34197
away_score    34197
tournament    34197
city          34197
country       34197
neutral       34197
id            34197
dtype: int64

Interesting, there is no null field in the dataset, so it will not be necessary to adjust the dataset to such values.

In [98]:
df_historical_results.describe(include='object')

,date,home_team,away_team,tournament,city,country
count,44060,44060,44060,44060,44060,44060
unique,15496,311,306,139,2003,267
top,2012-02-29,Brazil,Uruguay,Friendly,Kuala Lumpur,United States
freq,66,591,558,17425,675,1259


In [8]:
df_ranking.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,1,Germany,GER,57.0,0.0,0,UEFA,1992-12-31
1,96,Syria,SYR,11.0,0.0,0,AFC,1992-12-31
2,97,Burkina Faso,BFA,11.0,0.0,0,CAF,1992-12-31
3,99,Latvia,LVA,10.0,0.0,0,UEFA,1992-12-31
4,100,Burundi,BDI,10.0,0.0,0,CAF,1992-12-31


In [7]:
df_win_loose_draw.head()

,country1,country2,games,wins,looses,draws
0,Argentina,Australia,7,0.714286,0.142857,0.142857
1,Australia,Argentina,7,0.142857,0.714286,0.142857
2,Argentina,Belgium,4,0.750000,0.250000,0.000000
3,Belgium,Argentina,4,0.250000,0.750000,0.000000
4,Argentina,Brazil,108,0.361111,0.398148,0.240741


In [10]:
df_win_loose_draw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798 entries, 0 to 797
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   country1  798 non-null    object 
 1   country2  798 non-null    object 
 2   games     798 non-null    int64  
 3   wins      798 non-null    float64
 4   looses    798 non-null    float64
 5   draws     798 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 37.5+ KB


In [11]:
df_ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63916 entries, 0 to 63915
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rank             63916 non-null  int64  
 1   country_full     63916 non-null  object 
 2   country_abrv     63916 non-null  object 
 3   total_points     63916 non-null  float64
 4   previous_points  63916 non-null  float64
 5   rank_change      63916 non-null  int64  
 6   confederation    63916 non-null  object 
 7   rank_date        63916 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 3.9+ MB
